In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq -n ngrok-stable-linux-amd64.zip
!pip install flask-ngrok
!./ngrok authtoken 2ainFcf2qyBUllRP7POKymXmNi2_6vA1zUUP3bjP2EsCVKiyF
!pip install gTTS
!pip install flask_cors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
# 제육복음
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
import transformers
import string
import re
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from gtts import gTTS
import os
import time
import base64
import shutil
from tensorflow import keras
import numpy as np
from PIL import Image
import io
from flask_cors import CORS
from PIL import Image, ImageOps
from keras.models import load_model
# data = load_dataset("beomi/KoAlpaca-v1.1a")
data = load_dataset("mncai/MedGPT-5k-ko")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 5304
    })
})

In [5]:
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" })

Map:   0%|          | 0/5304 [00:00<?, ? examples/s]

In [6]:
# model_id = "beomi/KoAlpaca-Polyglot-5.8B"
model_id = "beomi/polyglot-ko-12.8b-safetensors"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

...

model-00026-of-00028.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00027-of-00028.safetensors:   0%|          | 0.00/896M [00:00<?, ?B/s]

model-00028-of-00028.safetensors:   0%|          | 0.00/518M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/100 [00:00<?, ?B/s]

In [7]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/5304 [00:00<?, ? examples/s]

In [8]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [9]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [10]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=600,
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

model.eval()
model.config.use_cache = True

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.745800
20,2.454700
30,2.032700
40,1.866100
50,1.918700
60,1.723600
70,1.543300
80,1.688900
90,1.497600
100,1.523800


In [ ]:
def gen(x):
    prompt = f"### 친한 친구의 일상적인 말: {x}\n\n### 친한 친구의 일상적인 말에 공감하며 대답해주는 친구의 답변:"

    gened = model.generate(
        **tokenizer(prompt, return_tensors='pt', return_token_type_ids=False),
        max_new_tokens=50,
        early_stopping=False,
        do_sample=True,
        temperature=0.5,
        top_k=3,
        eos_token_id=2,
    )
    response = tokenizer.decode(gened[0], skip_special_tokens=True)

    response = response.replace(prompt, "").strip()

    special_chars = set(string.punctuation) - set(['?', '!', '.', ','])
    for char in special_chars:
        if char in response:
            response = response.split(char)[0]
            break
    return response
gen("나 너무 아파")

In [ ]:
def gen_feedback(x):
    prompt = f"### 사용자는 말하는 연습을 하려고 하는 어린아이: {x}\n\n### 너는 사용자의 말에 문제점을 분석하고 피드백 해주는 AI야:"

    gened = model.generate(
        **tokenizer(prompt, return_tensors='pt', return_token_type_ids=False),
        max_new_tokens=30,
        early_stopping=False,
        do_sample=True,
        temperature=0.6,
        top_k=5,
        eos_token_id=2,
    )
    response = tokenizer.decode(gened[0], skip_special_tokens=True)

    response = response.replace(prompt, "").strip()

    special_chars = set(string.punctuation) - set(['?', '!', '.', ','])
    for char in special_chars:
        if char in response:
            response = response.split(char)[0]
            break
    return response
gen_feedback("응 알려줘")

In [ ]:
def gen_hospital(x):
    prompt = f"### 병원에 진료를 받으러온 환자: {x}\n\n### 환자의 말에 공감하며 친절하게 한 문장으로 답해주는 의사 선생님:"

    gened = model.generate(
        **tokenizer(prompt, return_tensors='pt', return_token_type_ids=False),
        max_new_tokens=50,
        early_stopping=False,
        do_sample=True,
        temperature=0.6,
        top_k=10,
        eos_token_id=2,
    )
    response = tokenizer.decode(gened[0], skip_special_tokens=True)

    response = response.replace(prompt, "").strip()

    special_chars = set(string.punctuation) - set(['?', '!', '.', ','])
    for char in special_chars:
        if char in response:
            response = response.split(char)[0]
            break
    return response
gen_hospital("어떤 검사를 해야하나요?")

## 몇 가지 팁

- 만약 학습이 충분히 되지 않으면 `<|endoftext|>` 토큰이 잘 생성되지 않을 수 있습니다
- 이럴떈 충분히 긴 `max_new_tokens`를 준 뒤, `###`으로 잘라서 써보세요. ex) `output.split('###')[0]`
- 아래 결과는 실제 위 학습된(50step, 100개 샘플) 모델의 결과물입니다.
- 생성시에 속도가 꽤 느립니다. 1-2tokens/s 정도라 256토큰 생성시 약 ~3분 시간이 소요됩니다.

#===============================================================================
# Flask part

In [ ]:
if not os.path.exists("./gen"):
    os.makedirs("./gen")

if not os.path.exists("./gen_feedback"):
    os.makedirs("./gen_feedback")

if not os.path.exists("./gen_hospital"):
    os.makedirs("./gen_hospital")

In [ ]:
np.set_printoptions(suppress=True)

model_img = load_model("/content/drive/MyDrive/converted_keras/keras_model.h5", compile=False)

class_names = open("/content/drive/MyDrive/converted_keras/labels.txt", "r").readlines()

In [ ]:
import tensorflow as tf

In [ ]:
# 대화
def gen(x, ch):
  if ch == 0:
    prompt = f"### 친한 친구의 일상적인 말: {x}\n\n### 친한 친구의 일상적인 말에 공감하며 대답해주는 친구의 답변:"
  elif ch == 1:
    prompt = f"### 친한 친구의 일상적인 말: {x}\n\n### 친한 친구의 일상적인 말에 공감하며 대답해주고 말 끝에 항상 😃이모지를 붙이는 친구:"
  else:
    return "읽기 불가"

  gened = model.generate(
        **tokenizer(prompt, return_tensors='pt', return_token_type_ids=False),
        max_new_tokens=30,
        early_stopping=False,
        do_sample=True,
        temperature=0.6,
        top_k=5,
        eos_token_id=2,
    )
  response = tokenizer.decode(gened[0], skip_special_tokens=True)

  response = response.replace(prompt, "").strip()

  special_chars = set(string.punctuation) - set(['?', '!', '.', ','])
  for char in special_chars:
      if char in response:
          response = response.split(char)[0]
          break
  return response


# 피드백
def gen_feedback(x):
    prompt = f"### 사람들과 말하는 능력을 키우려고 연습하는 친구: {x}\n\n### 친구의 말에 피드백 해주고 예시 답변을 이야기 해주며 말하는 연습을 하는 친구를 도와주는 친구:"

    gened = model.generate(
        **tokenizer(prompt, return_tensors='pt', return_token_type_ids=False),
        max_new_tokens=30,
        early_stopping=False,
        do_sample=True,
        temperature=0.6,
        top_k=5,
        eos_token_id=2,
    )
    response = tokenizer.decode(gened[0], skip_special_tokens=True)

    response = response.replace(prompt, "").strip()

    special_chars = set(string.punctuation) - set(['?', '!', '.', ','])
    for char in special_chars:
        if char in response:
            response = response.split(char)[0]
            break
    return response


# 상황극
def gen_hospital(x):
    prompt = f"### 병원에 진료를 받으러온 환자: {x}\n\n### 환자의 말에 공감하며 친절하게 한 문장으로 답해주는 의사 선생님:"

    gened = model.generate(
        **tokenizer(prompt, return_tensors='pt', return_token_type_ids=False),
        max_new_tokens=30,
        early_stopping=False,
        do_sample=True,
        temperature=0.6,
        top_k=5,
        eos_token_id=2,
    )
    response = tokenizer.decode(gened[0], skip_special_tokens=True)

    response = response.replace(prompt, "").strip()

    special_chars = set(string.punctuation) - set(['?', '!', '.', ','])
    for char in special_chars:
        if char in response:
            response = response.split(char)[0]
            break
    return response

app = Flask(__name__)
CORS(run_with_ngrok(app))

# 대화
@app.route('/ai/gen', methods=['POST'])
def gen_text():
    data = request.get_json()
    text = data.get('text')
    ch = int(text[-1])
    text = str(text[:-1])
    result = gen(text, ch)
    result = str(result)
    print(type(result))
    modified_result = re.sub(r"[^\w\s\.\?,]", "", result)
    language = 'ko'
    timestamp = int(time.time())
    filename = f"/content/gen/{timestamp}.mp3"
    myobj = gTTS(text=result, lang=language, slow=False)
    myobj.save(filename)
    with open(filename, "rb") as audio_file:
      encoded = base64.b64encode(audio_file.read()).decode('utf-8')
    return jsonify({"audio": encoded, "text": result})

# 피드백
@app.route('/ai/gen_feedback', methods=['POST'])
def gen_feedback_text():
    data = request.get_json()
    text = data.get('text')
    result = gen_feedback(text)
    modified_result = re.sub(r"[^\w\s\.\?,]", "", result)
    language = 'ko'
    timestamp = int(time.time())
    filename = f"/content/gen_feedback/{timestamp}.mp3"
    myobj = gTTS(text=modified_result, lang=language, slow=False)
    myobj.save(filename)
    with open(filename, "rb") as audio_file:
      encoded = base64.b64encode(audio_file.read()).decode('utf-8')
    return jsonify({"audio": encoded, "text": modified_result})


# 상황극
@app.route('/ai/gen_hospital', methods=['POST'])
def gen_hospital_text():
    data = request.get_json()
    text = data.get('text')
    result = gen_hospital(text)
    modified_result = re.sub(r"[^\w\s\.\?,]", "", result)
    language = 'ko'
    timestamp = int(time.time())
    filename = f"/content/gen_hospital/{timestamp}.mp3"
    myobj = gTTS(text=modified_result, lang=language, slow=False)
    myobj.save(filename)
    with open(filename, "rb") as audio_file:
      encoded = base64.b64encode(audio_file.read()).decode('utf-8')
    return jsonify({"audio": encoded, "text": modified_result})


def preprocess_image(base64_string):
    image_data = base64.b64decode(base64_string)
    image = Image.open(io.BytesIO(image_data))
    image = image.convert("RGB")
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    return np.expand_dims(normalized_image_array, axis=0)


# 시선처리
@app.route('/ai/img', methods=['POST'])
def process_text():
    image_data = request.json['image']
    preprocessed_image_eme = preprocess_image(image_data)
    prediction = model_img.predict(preprocessed_image_eme)
    result = np.argmax(prediction, axis=1)
    return {"key": f"{result}"}

if __name__ == '__main__':
    app.run()